# This kernel uses two methods to solve the problem, processing with Fastai gives better results, but I would recommend anyone to go through the full code and be able to understand the differences in those methods, which gives us such a drastic variation in score

## Importing all the necessary stuff

In [1]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import os

## This is the basic code for accessing the data

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/bluebook-for-bulldozers/ValidSolution.csv
/kaggle/input/bluebook-for-bulldozers/Machine_Appendix.csv
/kaggle/input/bluebook-for-bulldozers/TrainAndValid.csv
/kaggle/input/bluebook-for-bulldozers/random_forest_benchmark_test.csv
/kaggle/input/bluebook-for-bulldozers/Test.csv
/kaggle/input/bluebook-for-bulldozers/median_benchmark.csv
/kaggle/input/bluebook-for-bulldozers/Data Dictionary.xlsx
/kaggle/input/bluebook-for-bulldozers/Valid.csv
/kaggle/input/bluebook-for-bulldozers/train/Train.csv
/kaggle/input/bluebook-for-bulldozers/trainandvalid/TrainAndValid.csv
/kaggle/input/bluebook-for-bulldozers/valid/Valid.csv


## Getting the Training and the Testing data

In [3]:
train = pd.read_csv('/kaggle/input/bluebook-for-bulldozers/trainandvalid/TrainAndValid.csv')
test = pd.read_csv('/kaggle/input/bluebook-for-bulldozers/Test.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Finding the type of data in those files

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
SalesID                     412698 non-null int64
SalePrice                   412698 non-null float64
MachineID                   412698 non-null int64
ModelID                     412698 non-null int64
datasource                  412698 non-null int64
auctioneerID                392562 non-null float64
YearMade                    412698 non-null int64
MachineHoursCurrentMeter    147504 non-null float64
UsageBand                   73670 non-null object
saledate                    412698 non-null object
fiModelDesc                 412698 non-null object
fiBaseModel                 412698 non-null object
fiSecondaryDesc             271971 non-null object
fiModelSeries               58667 non-null object
fiModelDescriptor           74816 non-null object
ProductSize                 196093 non-null object
fiProductClassDesc          412698 non-null object
state                      

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 0 to 12456
Data columns (total 52 columns):
SalesID                     12457 non-null int64
MachineID                   12457 non-null int64
ModelID                     12457 non-null int64
datasource                  12457 non-null int64
auctioneerID                12457 non-null int64
YearMade                    12457 non-null int64
MachineHoursCurrentMeter    2129 non-null float64
UsageBand                   1834 non-null object
saledate                    12457 non-null object
fiModelDesc                 12457 non-null object
fiBaseModel                 12457 non-null object
fiSecondaryDesc             8482 non-null object
fiModelSeries               2006 non-null object
fiModelDescriptor           3024 non-null object
ProductSize                 6048 non-null object
fiProductClassDesc          12457 non-null object
state                       12457 non-null object
ProductGroup                12457 non-null object
Pr

In [6]:
train.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [8]:
train.describe(include='all')

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
count,4.126980e+05,412698.000000,4.126980e+05,412698.000000,412698.000000,392562.000000,412698.000000,1.475040e+05,73670,412698,...,102916,102261,102332,102261,102193,80712,81875,81877,71564,71522
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,4013,...,19,29,3,3,3,2,10,7,4,5
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medium,2/16/2009 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,None or Unspecified,PAT,None or Unspecified,Standard,Conventional
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35832,1932,...,82444,81539,85074,92924,86998,80692,39633,71447,70169,70774
mean,2.011161e+06,31215.181414,1.230061e+06,6947.201828,135.169361,6.585268,1899.049637,3.522988e+03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.080068e+06,23141.743695,4.539533e+05,6280.824982,9.646749,17.158409,292.190243,2.716993e+04,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.139246e+06,4750.000000,0.000000e+00,28.000000,121.000000,0.000000,1000.000000,0.000000e+00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.421898e+06,14500.000000,1.088593e+06,3261.000000,132.000000,1.000000,1985.000000,0.000000e+00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.645852e+06,24000.000000,1.284397e+06,4605.000000,132.000000,2.000000,1995.000000,0.000000e+00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.261012e+06,40000.000000,1.478079e+06,8899.000000,136.000000,4.000000,2001.000000,3.209000e+03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Since this file has a lot of columns, lets check the number of NaN(s).....

In [9]:
train.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

## Looking at the type of values in each column. Other columns can be checked by replacing the column names

In [10]:
train.fiProductClassDesc

0                  Wheel Loader - 110.0 to 120.0 Horsepower
1                  Wheel Loader - 150.0 to 175.0 Horsepower
2         Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...
3         Hydraulic Excavator, Track - 12.0 to 14.0 Metr...
4         Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...
                                ...                        
412693    Hydraulic Excavator, Track - 2.0 to 3.0 Metric...
412694    Hydraulic Excavator, Track - 3.0 to 4.0 Metric...
412695    Hydraulic Excavator, Track - 2.0 to 3.0 Metric...
412696    Hydraulic Excavator, Track - 2.0 to 3.0 Metric...
412697    Hydraulic Excavator, Track - 3.0 to 4.0 Metric...
Name: fiProductClassDesc, Length: 412698, dtype: object

## Normalizing the data

## This time we are using the log of the Sale Price

In [11]:
train['SalePrice'] = np.log(train.SalePrice)

## Looking at the description above and considering the number of NaNs in each column, we would be better to consider only the really important columns

In [12]:
features_to_consider = ['YearMade', 'datasource', 'state', 'fiBaseModel', 'fiProductClassDesc' , 'fiModelDesc']

### This function will take in the training and validation data, and output mean squared error. This function will basically tell us how distributed our data really is.

In [13]:
def model_score(model, X_trn, y_trn, X_val, y_val):
    model.fit(X_trn, y_trn)
    pred = model.predict(X_val)
    return np.sqrt(mse(pred, y_val))

## Data preparation and preprocessing

In [14]:
X = train[features_to_consider]
y = train.SalePrice

## We will be using the Label Encoder as the number of unique values in each column are a lot. If we were to use One-Hot Encoding, the number of columns would increase drastically

In [15]:
LabelEnc = LabelEncoder()
X['state']=LabelEnc.fit_transform(X.state)
X['fiBaseModel']= LabelEnc.fit_transform(X.fiBaseModel)
X['fiProductClassDesc']= LabelEnc.fit_transform(X.fiProductClassDesc)
X['fiModelDesc']= LabelEnc.fit_transform(X.fiModelDesc)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## The first model we use is the Linear Regression

In [17]:
model = LinearRegression()
model_score(model, X_train, y_train, X_test, y_test)

0.6795421463065718

## This is the Random Forest Regressor

In [18]:
model = RandomForestRegressor(max_depth=30, min_samples_split=20, n_estimators=110, n_jobs= -1)
model_score(model, X_train, y_train, X_test, y_test)

0.2787302423975774

## Let's use Multi Layered Perceptrons too

In [19]:
model = MLPRegressor(hidden_layer_sizes=(100), activation="relu", solver="adam", alpha=0.0001, verbose=True)
model_score(model, X_train, y_train, X_test, y_test)

Iteration 1, loss = 185.96402068
Iteration 2, loss = 0.66974025
Iteration 3, loss = 0.47391681
Iteration 4, loss = 0.74408266
Iteration 5, loss = 1.06021685
Iteration 6, loss = 1.03833608
Iteration 7, loss = 0.86797286
Iteration 8, loss = 1.00615340
Iteration 9, loss = 0.90244359
Iteration 10, loss = 0.80472834
Iteration 11, loss = 0.84396455
Iteration 12, loss = 0.95613938
Iteration 13, loss = 0.72915241
Iteration 14, loss = 0.87795576
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


1.084310540839215

## Looking at the above mse(s), we can safely assume that our data is spread out all over

# While looking at the different types of methods to solve these type of problems, I came accross a Deep Learning library called fastai.

## Let's try it out

## This is the basic installation to be able to use fastai.structured

In [20]:
!pip install git+https://github.com/fastai/fastai@2e1ccb58121dc648751e2109fc0fbf6925aa8887

  Cloning https://github.com/fastai/fastai (to revision 2e1ccb58121dc648751e2109fc0fbf6925aa8887) to /tmp/pip-req-build-p2tqef_h
  Running command git clone -q https://github.com/fastai/fastai /tmp/pip-req-build-p2tqef_h
  Running command git checkout -q 2e1ccb58121dc648751e2109fc0fbf6925aa8887
     |████████████████████████████████| 496.4MB 27kB/s 
  Created wheel for fastai: filename=fastai-0.7.0-cp36-none-any.whl size=118998 sha256=8245d4cb2b51687acefb0db8e0fa0bd8ec2f1f7c133a5405b9cd51e17b0861ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-_69pm02n/wheels/25/e9/9b/223e47f59c39f0bd393733db0f7efe8ec6f078d29c32198f9f
Successfully built fastai
ERROR: allennlp 0.9.0 requires flaky, which is not installed.
ERROR: allennlp 0.9.0 requires responses>=0.7, which is not installed.
ERROR: pytorch-transformers 1.1.0 has requirement torch>=1.0.0, but you'll have torch 0.3.1 which is incompatible.
ERROR: pytorch-pretrained-bert 0.6.2 has requirement torch>=0.4.1, but you'll have torch 0.3.1 w

## Importing(again!!!)

In [21]:
import os

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas_summary import DataFrameSummary
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from time import time


## These are the fastai imports
from fastai.imports import *
from fastai.structured import *

## Loading the data

In [22]:
data = pd.read_csv('/kaggle/input/bluebook-for-bulldozers/trainandvalid/TrainAndValid.csv', low_memory=False, parse_dates=["saledate"])
data.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ‘train_cats’ method is used for turning ‘string’ type columns into ‘category’ type columns

In [23]:
train_cats(data)
data.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### We will use add_datepart helper function to add columns relevant to a date in the salesdate column

In [24]:
add_datepart(data, 'saledate')

### Let's take a look at the data now

In [25]:
data.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,226,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,23,3,204,False,False,False,False,False,False,1248307200


### We can see that the salesdate column is now gone, and instead we get different columns with different details of the date like day, days of the week, day of the year, etc. all of which are good determining factors for the Sale Price of the product

### Now for some of the other useful columns in this dataset, we can see that UsageBand can influence the Sale Price
### We can map it into numbers manually, but we will use the astype() function here

In [26]:
data.UsageBand = data.UsageBand.astype('category')
data.UsageBand = data.UsageBand.cat.codes

### Usage band has been converted into int instead of string 

In [27]:
data.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,1,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,1,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,0,226,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,0,PC120-6E,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,2,S175,...,23,3,204,False,False,False,False,False,False,1248307200


### Lets normalize the data
### We will be using log for that

In [28]:
data['SalePrice'] = np.log(data['SalePrice'])
data['SalePrice'].head()

0    11.097410
1    10.950807
2     9.210340
3    10.558414
4     9.305651
Name: SalePrice, dtype: float64

### This looks much simpler now

### Lets take at the empty values in our table, and for that we will be using proc_df function
1. For continuous variables, it checks whether a column has missing values or not
2. If the column has missing values, it creates another column called columnname_na, which has 1 for missing and 0 for not missing
3. Simultaneously, the missing values are replaced with the median of the column
4. For categorical variables, pandas replaces missing values with -1. So proc_df adds 1 to all the values for categorical variables. Thus, we have 0 for missing while all othervalues are incremented by 1

In [29]:
finalData, Y, nas = proc_df(data, 'SalePrice')

In [30]:
finalData.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,999089,3157,121,3.0,2004,68.0,1,963,298,...,320,False,False,False,False,False,False,1163635200,False,False
1,1139248,117657,77,121,3.0,1996,4640.0,1,1745,529,...,86,False,False,False,False,False,False,1080259200,False,False
2,1139249,434808,7009,121,3.0,2001,2838.0,0,336,111,...,57,False,False,False,False,False,False,1077753600,False,False
3,1139251,1026470,332,121,3.0,2001,3486.0,0,3716,1381,...,139,False,False,False,False,False,False,1305763200,False,False
4,1139253,1057373,17311,121,3.0,2007,722.0,2,4261,1538,...,204,False,False,False,False,False,False,1248307200,False,False


### Looks like we got rid of all the empty values in the table

In [31]:
print(Y)
len(Y)

[11.09741 10.95081  9.21034 ...  9.43348  9.21034  9.4727 ]


412698

### We will use Random Forest Regressor

In [32]:
model = RandomForestRegressor(n_jobs=-1)

### Training the data and getting the score

In [33]:
model.fit(finalData, Y)
model.score(finalData, Y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9831626575156694

### This looks comparatively better, but let's try splitting the data 

### Splitting the data into training and testing

In [34]:
X_train, X_test, y_train, y_test = train_test_split(finalData, Y, test_size=0.33, random_state=42)

In [35]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8985556735780489

### Finally we have a values that we can agree with, this gives a good score as compared to the previous efforts

In [36]:
print(model.score(X_test, y_test) * 100)

89.85556735780489
